In [73]:
import requests
import pandas as pd
import numpy as np
from requests_html import HTMLSession
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import time
from openpyxl import load_workbook
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog

#link getting functions to be used later
#gets the google url for player gamelog searches
def get_source(url):

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

#scrapes the google links resulting from each player gamelog search
def scrape_google_link(link):

    response = get_source(link)
    if response == "<Response [200]>":
        print("Google rate limited.")
    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.',
                    'https://google.',
                    'https://webcache.googleusercontent.',
                    'http://webcache.googleusercontent.',
                    'https://policies.google.',
                    'https://support.google.',
                    'https://maps.google.')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)
    return links

def getPlayerGameLog(playerName):
    player_dict = players.get_players()

    # Use ternary operator or write function 
    # Names are case sensitive
    aPlayer_id = str([player for player in player_dict if player['full_name'] == playerName][0]['id'])
    time.sleep(0.6)
    playerGameLog = playergamelog.PlayerGameLog(player_id=aPlayer_id, season = '2021')
    #time.sleep(0.6)
    df_play_games = playerGameLog.get_data_frames()
    #display(df_play_games)
    return df_play_games

#Scraper part

#change xpath or css selectors to be for whatever sport you want
#loads driver download location and opens website for scraping
driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get("https://www.mybookie.ag/props-builder/")
time.sleep(2)
#selects the frame to get info out of
WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, "/html/body/div[2]/div[1]/div/main/section/div/section/article/div[2]/div[2]/div/div/div[2]/div/div/div[2]/iframe")))
time.sleep(2)
#selects the league
#/html/body/main/div[1]/div/div/div/div[1]/league-toggle/div/div/ul/li[change this element to reflect which league to pick]/a
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/main/div[1]/div/div/div/div[1]/league-toggle/div/div/ul/li[1]/a"))).click()
time.sleep(2)
#selects over/under tab
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/main/div[1]/div/div/div/div[2]/div/div/div[3]/div[1]/ul/li[3]/a"))).click()
time.sleep(2)

#closes tab to count how many there are
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/main/div[1]/div/div/div/div[2]/div/div/player-search/div/div[3]/div/div/div/browse-bets/div/ul/div/li[1]/p"))).click()
time.sleep(2)

totalpropfinder = driver.find_elements(By.XPATH, "/html/body/main/div[1]/div/div/div/div[2]/div/div/player-search/div/div[3]/div/div/div/browse-bets/div/ul/div")
totalpropfinder2 = totalpropfinder[0].text
total_props = totalpropfinder2.count("\n") + 2

all_prop_names = []
all_lines = []
range_finder = 0
totalpropcounter = 1

#goes through all the prop types available, ex. Passing Yards, Rushing Yards, etc.
for type_of_prop in range(1, total_props):
    #opens next tab
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/main/div[1]/div/div/div/div[2]/div/div/player-search/div/div[3]/div/div/div/browse-bets/div/ul/div/li[" + str(totalpropcounter) + "]/p"))).click()
    time.sleep(2)
    #gets the name of the prop type
    proptype = driver.find_elements(By.XPATH, "/html/body/main/div[1]/div/div/div/div[2]/div/div/player-search/div/div[3]/div/div/div/browse-bets/div/ul/div/li[" + str(totalpropcounter) + "]/p")

    #appends the names of the types to a list to be used as columns or something in final table
    for q in proptype:
        all_prop_names.append(q.text)

    #loops through the table data
    lines = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "/html/body/main/div[1]/div/div/div/div[2]/div/div/player-search/div/div[3]/div/div/div/browse-bets/div/ul/div/li[" + str(totalpropcounter) + "]/div")))
    time.sleep(2)
    for x in lines:
        x2 = x.text.split("\n")
        all_lines.append(x2)


    totalpropcounter = totalpropcounter + 1

driver.quit()

for l in all_lines:
    l.remove("PLAYER")
    l.remove("OVER")
    l.remove("  LINE")
    l.remove("  UNDER")

#separates into respective columns, list of lists for each, with each list within the bigger list denoting one stat, so playerlist[0] shows all player names for the first stat (normally points), playerlist[2] is for rebounds etc.
r = 0
playerlist = []
gamelist = []
linelist = []
overlist = []
underlist = []

for w in range(0, len(all_lines)):
    x = all_lines[r]
    playerlist.append(x[0::5])
    gamelist.append(x[1::5])
    linelist.append(x[2::5])
    overlist.append(x[3::5])
    underlist.append(x[4::5])
    r = r + 1

#gets all player names for that days props
all_player_names = []
dict_play_gamelog = {}
for name in playerlist:
    for i in name:
        if i not in all_player_names:
            #print(i)
            if i == "Nah'Shon Hyland":
                all_player_names.append(i)
                dict_play_gamelog[i] = playergamelog.PlayerGameLog(player_id='1630538', season = '2021')
            else:
                if i == 'Otto Porter':
                    i = 'Otto Porter Jr.'
                elif i == 'Robert Williams':
                    i = 'Robert Williams III'
                all_player_names.append(i)
                thisPlayGL = getPlayerGameLog(i)
                dict_play_gamelog[i] = thisPlayGL
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
print (dict_play_gamelog)

#gets the list of all links from the google searches using each player name from that days props
links_list = []
for each_name in all_player_names:
    each_name = each_name.replace(" ", "+")
    name2 = "https://www.google.com/search?q=site%3Ahttps%3A%2F%2Fbasketball.realgm.com%2F+" + each_name + "+gamelogs&biw=767&bih=736&sxsrf=APq-WBtFX9k-TkDj2cSWvxfqO0VFbRS93g%3A1643524680248&ei=SDL2YaCvDou-0PEP85yrkAs&ved=0ahUKEwig8_rF7tj1AhULHzQIHXPOCrIQ4dUDCA4&uact=5&oq=site%3Ahttps%3A%2F%2Fbasketball.realgm.com%2F+" + each_name + "+gamelogs&gs_lcp=Cgdnd3Mtd2l6EAM6BwgAEEcQsANKBAhBGABKBAhGGABQhgVYlhZgohpoAXACeACAAaUBiAHmDZIBBDEuMTGYAQCgAQHIAQjAAQE&sclient=gws-wiz"
    finallink = scrape_google_link(name2)
    links_list.append(finallink)

#gets all the links needed for stat pulling
gameloglink = []
for each_link_list in links_list:
    for link in each_link_list:
        link = link.replace("/NBA/2022", "")
        link = link.replace("/All", "")
        link = link.replace("/D-League", "")
        link = link.replace("/NBA", "")
        link = link.replace("/Special", "")
        link = link.replace("/2021", "")
        if "GameLogs" in link and "NBA" not in link and link[(len(link) - 1)].isnumeric() and "International" not in link:
            for i in all_player_names:
                #filters out apostrophes in names and other naming problems
                i = i.replace("'", "")
                i = i.replace(".", "")
                i = i.casefold()
                link = link.casefold()
                if i.replace(" ", "-") in link and link not in gameloglink:
                    gameloglink.append(link)

#sets pandas data frame to be able to view all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

#zips to get the stats in their respective columns then explodes to get rid of nested list
#all_lines_final = list(zip(*all_lines))
datatable = pd.DataFrame({"Type": all_prop_names, "Player": playerlist, "Game": gamelist, "Line": linelist, "Over": overlist, "Under": underlist})
datatable2 = datatable.set_index(["Type"]).apply(pd.Series.explode).reset_index()

#writes the prop table to excel sheet
book = load_workbook("NBA PYTHON.xlsx")
writer = pd.ExcelWriter("NBA PYTHON.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace")
book = writer.book
datatable2.to_excel("NBA PYTHON.xlsx", index=False)
#writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
datatable2.to_excel(writer, index=False)
writer.save()

#appends links column to workbook
linktable = pd.DataFrame({"Links": gameloglink})
book = load_workbook("NBA PYTHON.xlsx")
sheet = book['Sheet1']
for index, row in linktable.iterrows():
    cell = 'G%d' % (index + 2)
    sheet[cell] = row[0]
book.save("NBA PYTHON.xlsx")

print("Done")

""" TODO: Drop Date, use just order of most recent games (most recent largest), drop all ID's
drop their own team, keep only opponent team, home or away feature splitting by space's and if its an @ or a vs. 
opponent team to opponent team's defensive rating using a dictionary with the defensive rating values hardcoded
, drop video_available

Find out how to predict single feature

Build one neural network per player (the dataframe in dict_play_gamelog)
Then run on a player to predict points as an example feeding only the game recency (which would be larger than the largest recency value by 1),
opponent team, home or away. Get this data from either the excel file or just straight from the python dataframe 
"""




C:\Users\Jason\AppData\Local\Temp\ipykernel_53268\1136948745.py:63: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
C:\Users\Jason\AppData\Local\Temp\ipykernel_53268\1136948745.py:159: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


{'Al Horford': [   SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  FGA  FG_PCT  FG3M  FG3A  FG3_PCT  FTM  FTA  FT_PCT  OREB  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  VIDEO_AVAILABLE
0   22021     201143     0022101223  APR 10, 2022  BOS @ MEM    W  24   6    8    0.750   1     1     1.000    0    0    0.000   0     6     6    2    0    1    2    0   13   13          1              
1   22021     201143     0022101193  APR 06, 2022  BOS @ CHI    W  30   7    7    1.000   3     3     1.000    0    0    0.000   4     6     10   2    2    1    0    0   17   23          1              
2   22021     201143     0022101166  APR 03, 2022  BOS vs. WAS  W  30   4    9    0.444   2     6     0.333    0    0    0.000   1     7     8    6    1    1    0    2   10   34          1              
3   22021     201143     0022101153  APR 01, 2022  BOS vs. IND  W  34   7    8    0.875   2     2     1.000    1    1    1.000   3     7     10   6    0    1    1    5   17

" TODO: Drop Date, use just order of most recent games (most recent largest), drop all ID's\ndrop their own team, keep only opponent team, home or away feature splitting by space's and if its an @ or a vs. \nopponent team to opponent team's defensive rating using a dictionary with the defensive rating values hardcoded\n, drop video_available\n\nFind out how to predict single feature\n\nBuild one neural network per player (the dataframe in dict_play_gamelog)\nThen run on a player to predict points as an example feeding only the game recency (which would be larger than the largest recency value by 1),\nopponent team, home or away. Get this data from either the excel file or just straight from the python dataframe \n"

In [66]:
teamNumDict = {}

curTeam = 0

for player in dict_play_gamelog:
    for game in dict_play_gamelog[player]:
        match = game.loc[:, 'MATCHUP']
        for day in match:
            spl = day.split()
            for val in spl:
                if val != 'vs.' :
                    if val not in teamNumDict:
                        teamNumDict[val] = curTeam
                        curTeam += 1
                    
teamNumDict.pop('@')
teamNumDict['BOS']= 106.2
teamNumDict['MEM']= 108.9
teamNumDict['CHI']= 113.2
teamNumDict['WAS']= 113.6
teamNumDict['IND']= 115.5
teamNumDict['MIA']= 108.4
teamNumDict['UTA']= 110.0
teamNumDict['OKC']= 111.7
teamNumDict['DEN']= 111.5
teamNumDict['SAC']= 114.8
teamNumDict['GSW']= 106.6
teamNumDict['DAL']= 109.1
teamNumDict['DET']= 113.3
teamNumDict['CHA']= 113.1
teamNumDict['BKN']= 112.3
teamNumDict['ATL']= 113.7
teamNumDict['PHI']= 110.2
teamNumDict['ORL']= 112.1
teamNumDict['NOP']= 112.0
teamNumDict['POR']= 116.3
teamNumDict['NYK']= 110.2
teamNumDict['SAS']= 111.7
teamNumDict['PHX']= 106.8
teamNumDict['LAC']= 109.5
teamNumDict['MIN']= 111.0
teamNumDict['MIL']= 111.1
teamNumDict['LAL']= 112.8
teamNumDict['TOR']= 109.9
teamNumDict['HOU']= 116.2
teamNumDict['CLE']= 108.9
teamNumDict



     

{'BOS': 106.2,
 'MEM': 108.9,
 'CHI': 113.2,
 'WAS': 113.6,
 'IND': 115.5,
 'MIA': 108.4,
 'UTA': 110.0,
 'OKC': 111.7,
 'DEN': 111.5,
 'SAC': 114.8,
 'GSW': 106.6,
 'DAL': 109.1,
 'DET': 113.3,
 'CHA': 113.1,
 'BKN': 112.3,
 'ATL': 113.7,
 'PHI': 110.2,
 'ORL': 112.1,
 'NOP': 112.0,
 'POR': 116.3,
 'NYK': 110.2,
 'SAS': 111.7,
 'PHX': 106.8,
 'LAC': 109.5,
 'MIN': 111.0,
 'MIL': 111.1,
 'LAL': 112.8,
 'TOR': 109.9,
 'HOU': 116.2,
 'CLE': 108.9}

In [74]:
def matchupToNum(dpg):
    for player in dpg:
        for game in dpg[player]:
            match = game.loc[:, 'MATCHUP']
            defense = []
            site = []
            for day in match:
                spl = day.split()
                if 'vs.' in spl:
                    site.append(1)
                else:
                    site.append(0)
                defense.append(teamNumDict[spl[2]])
            game['Defense']= defense
            game['Home'] = site
                

                
    


In [75]:

x = dict_play_gamelog.get('Jayson Tatum')
x

[   SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  FGA  FG_PCT  FG3M  FG3A  FG3_PCT  FTM  FTA  FT_PCT  OREB  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  VIDEO_AVAILABLE
 0      22021    1628369  0022101223  APR 10, 2022    BOS @ MEM  W   26   11   14   0.786     6     8    0.750    3    3   1.000     0     9    9    3    1    0    5   2   31          29                1
 1      22021    1628369  0022101193  APR 06, 2022    BOS @ CHI  W   30    5   18   0.278     1     8    0.125    5    7   0.714     0    10   10    8    2    1    2   1   16          22                1
 2      22021    1628369  0022101166  APR 03, 2022  BOS vs. WAS  W   29    9   17   0.529     1     5    0.200    3    6   0.500     2     4    6    7    2    1    2   3   22          22                1
 3      22021    1628369  0022101153  APR 01, 2022  BOS vs. IND  W   37   10   24   0.417     3     9    0.333    8   10   0.800     2     4    6    6    1    1    1   0   31          

In [81]:
for player in dict_play_gamelog:
        for game in dict_play_gamelog[player]:
            '''game.drop('WL', axis =1, inplace=True)
            game.drop('Game_ID', axis =1, inplace=True)
            game.drop('GAME_DATE', axis =1, inplace=True)
            game.drop('VIDEO_AVAILABLE', axis =1, inplace=True)
            game.drop('SEASON_ID', axis= 1, inplace= True)
            game.drop('Player_ID', axis= 1, inplace= True)
            recent = []
            for i in range(len(game)):
                recent.append(len(game)-i)
                
            game['MostRecent']= recent'''
            game.drop('MATCHUP', axis= 1, inplace= True)



#matchupToNum(dict_play_gamelog)
                


KeyError: "['MATCHUP'] not found in axis"

In [79]:
for player in dict_play_gamelog:
        for game in dict_play_gamelog[player]:
            game.drop('WL', axis =1, inplace=True)
            game.drop('Game_ID', axis =1, inplace=True)
            game.drop('GAME_DATE', axis =1, inplace=True)
            game.drop('VIDEO_AVAILABLE', axis =1, inplace=True)
            game.drop('SEASON_ID', axis= 1, inplace= True)
            game.drop('Player_ID', axis= 1, inplace= True)
            game.drop('MATCHUP', axis= 1, inplace= True)
            recent = []
            for i in range(len(game)):
                recent.append(len(game)-i)

KeyError: "['WL'] not found in axis"

In [82]:
x

[    MIN  FGM  FGA  FG_PCT  FG3M  FG3A  FG3_PCT  FTM  FTA  FT_PCT  OREB  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  MostRecent  Defense  Home
 0    26   11   14   0.786     6     8    0.750    3    3   1.000     0     9    9    3    1    0    5   2   31          29          76    108.9     0
 1    30    5   18   0.278     1     8    0.125    5    7   0.714     0    10   10    8    2    1    2   1   16          22          75    113.2     0
 2    29    9   17   0.529     1     5    0.200    3    6   0.500     2     4    6    7    2    1    2   3   22          22          74    113.6     1
 3    37   10   24   0.417     3     9    0.333    8   10   0.800     2     4    6    6    1    1    1   0   31          -1          73    115.5     1
 4    40    9   17   0.529     0     5    0.000    5    6   0.833     1     4    5    6    2    0    4   4   23         -12          72    108.4     1
 5    34   12   21   0.571     5    11    0.455    5    5   1.000     0     5    5    6    1  

In [91]:

models = {}
player = 'Al Horford'
#for player in dpg:
for game in dict_play_gamelog[player]:


    A = game.to_numpy()   
    A = A.astype('float64')  # so many:  www.tutorialspoint.com/numpy/numpy_data_types.htm

    X_all = A[:,[20,21,22]]  # X (features) ... is all rows, columns 0, 1, 2, 3
    y_all = A[:,18]
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)

    USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

    # we "train the scaler"  (computes the mean and standard deviation)
    if USE_SCALER == True:
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        scaler.fit(X_train)  # Scale with the training data! ave becomes 0; stdev becomes 1
    else:
        # this one does no scaling!  We still create it to be consistent:
        scaler = StandardScaler(copy=True, with_mean=False, with_std=False) # no scaling
        scaler.fit(X_train)  # still need to fit, though it does not change...

    scaler   # is now defined and ready to use...

    # ++++++++++++++++++++++++++++++++++++++++++++++++++++++

    # Here are our scaled training and testing sets:

    X_train_scaled = scaler.transform(X_train) # scale!
    X_test_scaled = scaler.transform(X_test) # scale!

    y_train_scaled = y_train  # the predicted/desired labels are not scaled
    y_test_scaled = y_test  # not using the scaler

    def ascii_table(X,y):
        """ print a table of binary inputs and outputs """
        print(f"{'input ':>58s} -> {'pred':<5s} {'des.':<5s}") 
        for i in range(len(y)):
            print(f"{X[i,:]!s:>58s} -> {'?':<5s} {y[i]:<5.0f}")   # !s is str ...

    from sklearn.neural_network import MLPClassifier

    nn_classifier = MLPClassifier(hidden_layer_sizes=(6,7),  # 3 input -> 6 -> 7 -> 1 output
                        max_iter=500,      # how many times to train
                        activation="tanh", # the "activation function" input -> output
                        solver='sgd',      # the algorithm for optimizing weights
                        verbose=True,      # False to "mute" the training
                        shuffle=True,      # reshuffle the training epochs?
                        random_state=None, # set for reproduceability
                        learning_rate_init=.1,       # learning rate: % of error to backprop
                        learning_rate = 'adaptive')

    models[player] = nn_classifier
                        
        # soften feedback as it converges

    # documentation:
    # scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html 
    #     Try verbose / activation "relu" / other network sizes ...

    print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
    nn_classifier.fit(X_train_scaled, y_train_scaled)
    print("\n++++++++++  TRAINING:   end  +++++++++++++++")
    print(f"The analog prediction error (the loss) is {nn_classifier.loss_}")
                    


                #
                # Note that the zeros have become -1's
                # and the 1's have stayed 1's
                #



++++++++++  TRAINING:  begin  +++++++++++++++


Iteration 1, loss = 2.98429564
Iteration 2, loss = 2.96802514
Iteration 3, loss = 2.94579837
Iteration 4, loss = 2.91910650
Iteration 5, loss = 2.88934063
Iteration 6, loss = 2.85772644
Iteration 7, loss = 2.82529048
Iteration 8, loss = 2.79285323
Iteration 9, loss = 2.76104185
Iteration 10, loss = 2.73031456
Iteration 11, loss = 2.70098955
Iteration 12, loss = 2.67327238
Iteration 13, loss = 2.64727799
Iteration 14, loss = 2.62304523
Iteration 15, loss = 2.60054466
Iteration 16, loss = 2.57968281
Iteration 17, loss = 2.56030728
Iteration 18, loss = 2.54221674
Iteration 19, loss = 2.52517788
Iteration 20, loss = 2.50894819
Iteration 21, loss = 2.49330089
Iteration 22, loss = 2.47804692
Iteration 23, loss = 2.46304991
Iteration 24, loss = 2.44823174
Iteration 25, loss = 2.43356897
Iteration 26, loss = 2.41908212
Iteration 27, loss = 2.40482082
Iteration 28, loss = 2.39084778
Iteration 29, loss = 2.37722408
Iteration 30, loss = 2.36399745

C:\Users\Jason\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [92]:

def make_prediction( Features, nn, scaler ):
    """ uses nn for predictions """
    print("input features are", Features)
    #  we make sure Features has the right shape (list-of-lists)
    row = np.array( [Features] )  # makes an array-row
    row = scaler.transform(row)   # scale according to scaler
    print("nn.predict_proba == ", nn.predict_proba(row))   # probabilities of each
    prediction = nn.predict(row)  # max!
    return prediction
    
# our features -- note that the inputs don't have to be bits!
Features = [77 ,  111.1 , 1]     # whatever we'd like to test
prediction = make_prediction(Features, nn_classifier, scaler)
print(f"prediction: {prediction}")   

input features are [77, 111.1, 1]
nn.predict_proba ==  [[1.93691506e-05 7.41812184e-03 4.47743541e-04 4.56771250e-04
  4.00009397e-02 1.01174878e-04 7.22492475e-04 5.58526358e-05
  7.44501427e-01 1.72699872e-02 2.58144558e-07 2.71831985e-04
  3.27219402e-06 6.67716226e-06 5.31115336e-08 1.86931832e-01
  3.37335982e-06 3.55203710e-04 5.30767400e-08 1.43356546e-03]]
prediction: [10.]


### TODO List:
1. Turn prediction/neural network into functions and loop over each player available
2. Compare prediction and the betting lines and output either Over / Under
3. Create dictionary of nicknames for players in case nba-api doesn't request properly
4. Find more features to use in the prediction/neural network